# FusionBase_UNODC generic script:

### Directories creation:

In [1]:
# Networking
import requests

from fb_country.utils import get_country_attribute



# File handling
import zipfile
import os
import json

# Data wrangling
import pandas as pd
import re
import numpy as np
import hashlib

from collections import Counter

# Dateteime
from datetime import datetime
#update
from apscheduler.schedulers.blocking import BlockingScheduler


#GenericDirectories
import os
from pathlib import Path
import shutil

home_directory = str(Path.home())
print(home_directory)
DownloadedFilesDirectory=home_directory+'/FBDirectory/'
ExportedFilesDirectory=home_directory+'/TransformedFilesDirectory/'
FinalDirectory=home_directory+'/FBFinalDirectory/'
# define the name of the directory to be created
Directories=[DownloadedFilesDirectory,ExportedFilesDirectory,FinalDirectory]
for d in Directories:
    if not(os.path.exists(d)):
        try:
            os.mkdir(d)
            print("succesfully created "+d)
        except OSError:
            print ("Creation of the directory failed " +d)
    else:
        print ("Directory exists already " +d)

C:\Users\Adem
succesfully created C:\Users\Adem/FBDirectory/
succesfully created C:\Users\Adem/TransformedFilesDirectory/
succesfully created C:\Users\Adem/FBFinalDirectory/


### Automated Tasks:

In [ ]:
#Schedule the automated tasks to be done regularly for all possible data sources
def update():
    def _get_fb_id(row):
        '''
        Private util method to construct fb_id (fusionbase id) by hashing row values.
        '''
        concat_str = ''.join([str(x) for x in np.array(row)])

        return hashlib.sha3_256(concat_str.encode('utf-8')).hexdigest()[:32]

    TemporaryDirectory=home_directory+'/FBtempDirectory/'
    try:
        os.mkdir(TemporaryDirectory)
        print("succesfully created "+TemporaryDirectory)
    except OSError:
        print ("Creation of the directory failed "+TemporaryDirectory)
        #Downloading data from sources
    url=['car_theft.xlsx','theft_0.xlsx','bulglary.xlsx','money_laundering.xlsx','fraud.xlsx','cjs_court.xlsx','cjs_prosecution.xlsx','cjspolicepersonnel.xlsx','cjsp_convicted.xlsx','cjsp_prosecuted.xlsx','cjsp_fromal_contact.xlsx','personheld-mortality.xlsx','personsheld-unsentenced-length.xlsx','unsentenced.xlsx','total_unsentenced.xls','personsheld.xlsx','persons_held_total.xls','sexual_exploitation_0.xlsx','sexual_violence_0.xlsx','kidnapping_0.xlsx','robbery_0.xlsx','serious_assault_0.xlsx','armsseizedfoundsurrenderedbytracing.csv_.xlsx','wholesales-price.xlsx','drugrelateddeath.xlsx','prisionfacilities.xlsx','prison_capacity.xls','prisonstaff.xlsx','personsenteringprisons.xlsx','personheld-nationality.xlsx','intentional_homicide-bysexandage.xls','homicide-citizenship.xls','crime_related_situational_context.xls','ipfm.xls','homicidebyagesex.xlsx',
         'peopleconvicted-tip.xlsx','detected-trafficking.xlsx','homicide_sex_citizenship.xls','city-homicide.xls','armsseizedbytype.xlsx','treatment.xlsx']   

    try:
        for u in url:
            r = requests.get('https://dataunodc.un.org/sites/dataunodc.un.org/files/'+u, allow_redirects=True)  
            open(DownloadedFilesDirectory+u, 'wb').write(r.content)
    except Exception:
        print('Could not download data') 

        #Extracting data from excel files
    listcol_count=[]
    for u in url:
        df = pd.read_excel(DownloadedFilesDirectory+u, index_col=None, header=None)
        #Calculating the number of columns and indexes and stock them in lists to distinguish columns and indexes 
        #by trying to measure the length of distinct values and compare it to the length of columns

        col_count=0
        ind_count=0
        AllList=[[] for i in range(len(df))]
        columnsList=[[] for i in range(len(df))]
        indexList=[[] for i in range(len(df))]
        for j in range(len(df.columns)):
            for i in range(len(df)):

                if((type(df.iloc[i,j])!=int and type(df.iloc[i,j])!=float and str(df.iloc[i,j])!='nan' 
                    and type(df.iloc[i,j])!=np.float64) or (i==0 and str(df.iloc[i,j])!='nan')):

                        AllList[i].append(df.iloc[i,j])



                if((type(df.iloc[i,j])!=int and type(df.iloc[i,j])!=float and str(df.iloc[i,j])!='nan')
                    and type(df.iloc[i,j])!=np.float64):
                        columnsList[j].append(df.iloc[i,j])
                        if (len(columnsList[j])>len(df)/2 ):
                            if (0<=i<len(df)-1):
                                if (type(df.iloc[i+1,j])!=int and type(df.iloc[i+1,j])!=float):
                                    col_count=j+1

                            else: 
                                col_count=j+1

                        indexList[i].append(df.iloc[i,j])
                        if (len(indexList[i])>len(df.columns)/2):
                            ind_count=i+1
            #determine the indexes,the additional indexes, the columns and their names
        if ind_count==0:
            ind_count=1        
        names=[]
        additionalindex=[]
        columns=[]
        indexList=AllList[col_count:]
        for i in range(ind_count):
            if(i==0):

                if(len(AllList[i])==len(df.columns)):
                    additionalindex.append(list(Counter(AllList[0]).keys())[col_count:])
                    names.append(AllList[i][:col_count])

                else:
                    additionalindex.append(list(Counter(AllList[0]).keys()))

            elif(i==ind_count-1 and len(AllList[0])!=len(df)):
                columns.append(AllList[i][col_count:])
                names.append(AllList[i][:col_count])
            elif(1<len(Counter(df.iloc[i]).keys()) <4 ):
                additionalindex.append(AllList[i])

        tuples=AllList[ind_count:]
        tuplesfinal=[]
        for i in range(len(tuples)):
            for j in range(len(additionalindex)):
                for k in range(len(additionalindex[j])):
                    tuplesfinal.append(tuples[i]+[additionalindex[j][k]])

        namesfinal=[]
        for i in range(len(names)):
            for j in range(len(additionalindex)):
                namesfinal.append(names[i]+[j])
        columnsFinal=[]
        if columns:
            columnsFinal=list(Counter(columns[0]).keys())
            #determine the values to be diplayed
        values=[]
        if columnsFinal:
            for i in range(ind_count,len(df)):
                for j in range(col_count,len(df.columns)-1,len(columnsFinal)):
                    row=[]
                    for k in range(len(columnsFinal)):
                        row.append(df.iloc[i,j+k])
                    values.append(row)
        else:
            for i in range(ind_count,len(df)):
                for j in range(col_count,len(df.columns),1):
                    values.append(df.iloc[i,j])           
                #transform the data to a pivot table
        index = pd.MultiIndex.from_tuples(tuple(map(tuple,tuplesfinal)), names=namesfinal[0])

        if columns:

            df = pd.DataFrame(values, index=index, columns=list(Counter(columns[0]).keys()))
        else:
            df = pd.DataFrame(values, index=index)

        listcol_count.append(col_count)
        df.to_excel(TemporaryDirectory+u)
        df=pd.read_excel(TemporaryDirectory+u)
        date_time=[]
                #Adding the datetime and country_iso_alpha3
        for i in range(len(df)):
            date_time.append(datetime.now())
        df['fb_datetime']=date_time

        alpha=[]
        df['Country'] = df['Country'].astype(str)
        for i in range(len(df['Country'])):
            a=get_country_attribute(df['Country'][i],'alpha3')
            alpha.append(a)
        df.insert(col_count, 'country_iso_alpha3',alpha)

        fb_id=[]
        fb_id =  np.array([_get_fb_id(row) for row in df.values])
        df.set_index(fb_id,inplace=True)
        df.index.set_names('fb_id',inplace=True)
        #store the transformed data in destination
        df.columns = df.columns.astype(str)
        df.to_parquet(ExportedFilesDirectory+u+'.parquet.gzip',compression='gzip',engine='fastparquet',index=True)
    shutil.rmtree(TemporaryDirectory)
#Transforming datasets which contains 2 levels of additional index
    parquetIndlevel2=['car_theft.xlsx','theft_0.xlsx','bulglary.xlsx','money_laundering.xlsx','fraud.xlsx','cjs_court.xlsx','cjs_prosecution.xlsx','cjspolicepersonnel.xlsx','cjsp_convicted.xlsx','cjsp_prosecuted.xlsx','cjsp_fromal_contact.xlsx','personheld-mortality.xlsx','personsheld-unsentenced-length.xlsx','unsentenced.xlsx','total_unsentenced.xls','personsheld.xlsx','persons_held_total.xls','sexual_exploitation_0.xlsx','sexual_violence_0.xlsx','kidnapping_0.xlsx','robbery_0.xlsx','serious_assault_0.xlsx','armsseizedfoundsurrenderedbytracing.csv_.xlsx','wholesales-price.xlsx','drugrelateddeath.xlsx']
    #Transforming datasets which contains  
    for parquet, c in zip(parquetIndlevel2,listcol_count[:25]):

        df=pd.read_parquet(ExportedFilesDirectory+parquet+'.parquet.gzip')
        temp=df.iloc[0,0:c+1]
        for i in range(len(df)):
            for k in range(c+1):
                if (df.iloc[i,k]!=None and df.iloc[i,k]!="nan"):
                    temp[k]=df.iloc[i,k]
            for j in range(c+1):
                df.iloc[i,j]=temp[j]
        df.rename(columns={"0": "year"},inplace=True)
        df.to_parquet(FinalDirectory+parquet+'.parquet.gzip',compression='gzip',engine='fastparquet',index=True)
        #Transforming datasets which contains 1 levels of additional index
    parquetIndlevel1=['prisionfacilities.xlsx','prison_capacity.xls','prisonstaff.xlsx','personsenteringprisons.xlsx','personheld-nationality.xlsx','intentional_homicide-bysexandage.xls','homicide-citizenship.xls','crime_related_situational_context.xls','ipfm.xls','homicidebyagesex.xlsx',
         'peopleconvicted-tip.xlsx','detected-trafficking.xlsx','homicide_sex_citizenship.xls','city-homicide.xls','armsseizedbytype.xlsx','treatment.xlsx']
    convicted=[]
    trafficking=[]
    number_city_homicide=[]
    number_arm_seized=[]
    number_treatment=[]
    for parquet, c in zip(parquetIndlevel1,listcol_count[25:]):

        if (parquet=='peopleconvicted-tip.xlsx'):
            df=pd.read_parquet(ExportedFilesDirectory+'peopleconvicted-tip.xlsx.parquet.gzip')                       
            for i in range(len(df)):
                convicted.append('number of convicted people')
            df.insert(loc=4, column='Indicator', value=convicted)
            temp=df.iloc[0,0:c+1]
            for i in range(len(df)):
                for k in range(c+1):
                    if (df.iloc[i,k]!=None and df.iloc[i,k]!="nan"):
                        temp[k]=df.iloc[i,k]
                for j in range(c+1):
                    df.iloc[i,j]=temp[j]
            df.rename(columns={"0": "year","0.1":"Indicator value" },inplace=True)
            df.to_parquet(FinalDirectory+parquet+'.parquet.gzip',compression='gzip',engine='fastparquet',index=True)

        elif (parquet=='detected-trafficking.xlsx'):
            df=pd.read_parquet(ExportedFilesDirectory+'detected-trafficking.xlsx.parquet.gzip')                       
            for i in range(len(df)):
                trafficking.append('number of detected trafficking')
            df.insert(loc=4, column='Indicator', value=trafficking)
            temp=df.iloc[0,0:c+1]
            for i in range(len(df)):
                for k in range(c+1):
                    if (df.iloc[i,k]!=None and df.iloc[i,k]!="nan"):
                        temp[k]=df.iloc[i,k]
                for j in range(c+1):
                    df.iloc[i,j]=temp[j]
            df.rename(columns={"0": "year","0.1":"Indicator value" },inplace=True)
            df.to_parquet(FinalDirectory+parquet+'.parquet.gzip',compression='gzip',engine='fastparquet',index=True)

        elif(parquet =='armsseizedbytype.xlsx'):
            df=pd.read_parquet(ExportedFilesDirectory+'armsseizedbytype.xlsx.parquet.gzip')                       
            for i in range(len(df)):
                number_arm_seized.append('number of seized arms')
            df.insert(loc=8, column='Indicator', value=number_arm_seized)
            temp=df.iloc[0,0:c+1]
            for i in range(len(df)):
                for k in range(c+1):
                    if (df.iloc[i,k]!=None and df.iloc[i,k]!="nan"):
                        temp[k]=df.iloc[i,k]
                for j in range(c+1):
                    df.iloc[i,j]=temp[j]
            df.rename(columns={"0": "year","0.1":"Indicator value" },inplace=True)              
            df.to_parquet(FinalDirectory+parquet+'.parquet.gzip',compression='gzip',engine='fastparquet',index=True)

        elif (parquet =='homicide_sex_citizenship.xls'):
            df=pd.read_parquet(ExportedFilesDirectory+'homicide_sex_citizenship.xls.parquet.gzip')   
            df.rename(columns={"Variable": "Indicator"},inplace=True)

            temp=df.iloc[0,0:c+1]
            for i in range(len(df)):
                for k in range(c+1):
                    if (df.iloc[i,k]!=None and df.iloc[i,k]!="nan"):
                        temp[k]=df.iloc[i,k]
                for j in range(c+1):
                    df.iloc[i,j]=temp[j]
            df.rename(columns={"0": "year","0.1":"Indicator value" },inplace=True)
            df.to_parquet(FinalDirectory+parquet+'.parquet.gzip',compression='gzip',engine='fastparquet',index=True)

        elif(parquet=='city-homicide.xls'):
            df=pd.read_parquet(ExportedFilesDirectory+'city-homicide.xls.parquet.gzip')                       
            for i in range(len(df)):
                number_city_homicide.append('number of homicide')
            df.insert(loc=5, column='Indicator', value=number_city_homicide)
            temp=df.iloc[0,0:c+1]
            for i in range(len(df)):
                for k in range(c+1):
                    if (df.iloc[i,k]!=None and df.iloc[i,k]!="nan"):
                        temp[k]=df.iloc[i,k]
                for j in range(c+1):
                    df.iloc[i,j]=temp[j]
            df.rename(columns={"0": "year","0.1":"Indicator value" },inplace=True)              
            df.to_parquet(FinalDirectory+parquet+'.parquet.gzip',compression='gzip',engine='fastparquet',index=True)



        elif(parquet=='treatment.xlsx'):
            df=pd.read_parquet(ExportedFilesDirectory+'treatment.xlsx.parquet.gzip')                       
            for i in range(len(df)):
                number_treatment.append('number of treatment')
            df.insert(loc=4, column='Indicator', value=number_treatment)
            temp=df.iloc[0,0:c+1]
            for i in range(len(df)):
                for k in range(c+1):
                    if (df.iloc[i,k]!=None and df.iloc[i,k]!="nan"):
                        temp[k]=df.iloc[i,k]
                for j in range(c+1):
                    df.iloc[i,j]=temp[j]
            df.rename(columns={"0": "year","0.1":"Indicator value" },inplace=True)              
            df.to_parquet(FinalDirectory+parquet+'.parquet.gzip',compression='gzip',engine='fastparquet',index=True)
        else:
            df=pd.read_parquet(ExportedFilesDirectory+parquet+'.parquet.gzip')
            temp=df.iloc[0,0:c+1]
            for i in range(len(df)):
                for k in range(c+1):
                    if (df.iloc[i,k]!=None and df.iloc[i,k]!="nan"):
                        temp[k]=df.iloc[i,k]
                for j in range(c+1):
                    df.iloc[i,j]=temp[j]
            df.rename(columns={"0": "year","0.1":"Indicator value" },inplace=True)
            df.to_parquet(FinalDirectory+parquet+'.parquet.gzip',compression='gzip',engine='fastparquet',index=True)
            

#scheduler = apscheduler.schedulers.blocking.BackgroundScheduler('apscheduler.job_defaults.max_instances':'2')
scheduler = BlockingScheduler()
scheduler.add_job(update, 'interval', minutes=30,max_instances=5)
scheduler.start()


succesfully created C:\Users\Adem/FBtempDirectory/
succesfully created C:\Users\Adem/FBtempDirectory/
succesfully created C:\Users\Adem/FBtempDirectory/
